## Unpacking - Front Door Confounded
---

In [1]:
import gurobipy as gp
from gurobipy import GRB
import itertools
import numpy as np

model = gp.Model()
model.params.NonConvex = 2

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09
Set parameter NonConvex to value 2


In [2]:
(cardZ, cardA, cardM, cardY) = (2,2,2,2)

sample probabilitiy distribution generation of P_AMY|Z:  
irrelevant to rest of code

In [3]:
# Define the conditional probabilities
P_A_given_Z = {0: 0.3, 1: 0.7}  # Probability of A given Z
P_M_given_A = {0: 0.4, 1: 0.6}  # Probability of M given A
P_Y_given_M = {0: 0.2, 1: 0.8}  # Probability of Y given M
P_Z = {0: 0.5, 1: 0.5}         # Probability of Z

# Generate all combinations of Z, A, M, Y
combinations = list(itertools.product([0, 1], repeat=4))

# Calculate the joint probability distribution
P_ZAMY = {}
for z, a, m, y in combinations:
    P_ZAMY[(z, a, m, y)] = P_Z[z] * \
                           (P_A_given_Z[z] if a == 1 else 1 - P_A_given_Z[z]) * \
                           (P_M_given_A[a] if m == 1 else 1 - P_M_given_A[a]) * \
                           (P_Y_given_M[m] if y == 1 else 1 - P_Y_given_M[m])

# Normalize the distribution to ensure it sums to 1
total_prob = sum(P_ZAMY.values())
P_ZAMY_normalized = {k: v / total_prob for k, v in P_ZAMY.items()}


In [4]:
# rounding
P_ZAMY_normalized = {k: round(v, 3) for k, v in P_ZAMY_normalized.items()}

In [5]:
# Initialize dictionary to store P(Z)
P_Z_calculated = {0: 0, 1: 0}

# Calculate P(Z) for each value of Z by summing over all A, M, Y combinations
for z in [0, 1]:
    P_Z_calculated[z] = sum(P_ZAMY_normalized[(z, a, m, y)] for a, m, y in itertools.product([0, 1], repeat=3))

# Calculate P_AMY|Z as a new dictionary, ensuring no division by zero
P_AMY_given_Z = {}
for z in [0, 1]:
    # Check to prevent division by zero
    for a, m, y in itertools.product([0, 1], repeat=3):
        joint_prob = P_ZAMY_normalized[(z, a, m, y)]
        conditional_prob = joint_prob / P_Z_calculated[z]
        P_AMY_given_Z[(a, m, y, z)] = conditional_prob

P_AMY_giv_Z = P_AMY_given_Z

In [6]:
P_AMY_giv_Z

{(0, 0, 0, 0): 0.336,
 (0, 0, 1, 0): 0.084,
 (0, 1, 0, 0): 0.056,
 (0, 1, 1, 0): 0.224,
 (1, 0, 0, 0): 0.096,
 (1, 0, 1, 0): 0.024,
 (1, 1, 0, 0): 0.036,
 (1, 1, 1, 0): 0.144,
 (0, 0, 0, 1): 0.144,
 (0, 0, 1, 1): 0.036,
 (0, 1, 0, 1): 0.024,
 (0, 1, 1, 1): 0.096,
 (1, 0, 0, 1): 0.224,
 (1, 0, 1, 1): 0.056,
 (1, 1, 0, 1): 0.084,
 (1, 1, 1, 1): 0.336}

In [7]:
# observable_probs = np.arange(cardZ * cardA * cardM * cardY)
# observable_probs = observable_probs / observable_probs.sum()
# observable_probs = observable_probs.reshape(cardZ, cardA, cardM, cardY)


# P_ZAMY = {}
# # defining a p distribution based on cardinality
# for z, a, m, y in itertools.product(range(cardZ), range(cardA), range(cardM), range(cardY)):    
#     prob = observable_probs[z, a, m, y]
#     P_ZAMY[(z, a, m, y)] = prob

# # given distribution P_ZAMY get P_AMY|Z:
# P_AMY_giv_Z = {}
# for z, a, m, y in itertools.product(range(cardZ), range(cardA), range(cardM), range(cardY)):    
#     prob = P_ZAMY[(z, a, m, y)]
#     P_AMY_giv_Z[(z, a, m, y)] = prob / observable_probs[z, :, :, :].sum()

# print(sum(P_ZAMY.values()) == 1)

## consistency requirement

declaring mvar $Q(A_{Z=0}= , A_{Z=1} = , M_{A=0} = , M_{A=1}, Y_{M=0} = , Y_{M=1})$:

In [8]:
Q_AzAz_MaMa_YmYm = model.addMVar(shape = (cardA, cardA, cardM, cardM, cardY, cardY), 
                             vtype=GRB.CONTINUOUS, name="Q_(A0,A1,M0,M1,Y0,Y1)", lb=0, ub=1)

In [9]:
# used to print constraint names in a readable way

# works by listing symbolic variable name for the variables that are iterated through in the same for all possibilities,
# and an actual value for the fixed variables
def print_pretty(a, m, y, z):
    if z == 0:
        posA = 0
    elif z == 1:
        posA = 1
    
    if a == 0:
        posM = 2
    elif a == 1:
        posM = 3

    if m == 0:
        posY = 4
    elif m == 1:
        posY = 5

    lst = ['a', 'a', 'm', 'm', 'y', 'y']
    # lst = [':', ':', ':', ':', ':', ':']
    lst[posA] = a
    lst[posM] = m
    lst[posY] = y

    return lst

# m.addConstr(P_AMY_giv_Z[(0, 1, 0, 1)] == sum([Q_AzAz_MaMa_YmYm[:,0,1,:,:,0]]).sum())    

In [10]:
# coordinate finder used for P(a,m,y|z) = Σ_(a',m',y')[Q_(a,a',m,m',y,y')]
def Q_pos_from_P(a, m, y, z):
    if z == 0:
        posA0 = a
        posA1 = slice(None)  # Represents ':'
    else:  # z == 1
        posA0 = slice(None)
        posA1 = a
    
    if a==0:
        posM0 = m
        posM1 = slice(None)
    elif a==1:
        posM0 = slice(None)
        posM1 = m

    if m==0:
        posY0 = y
        posY1 = slice(None)
    elif m==1:
        posY0 = slice(None)
        posY1 = y

    # Creating a tuple that properly represents the slicing
    return (posA0, posA1, posM0, posM1, posY0, posY1)

adding constraints:  
$P(A=a, M=m, Y=y | Z = z) = \sum Q(A_{Z=0}= , A_{Z=1} = , M_{A=0} = , M_{A=1}, Y_{M=0} = , Y_{M=1})$

In [11]:
# consistency constraints
for a, m, y, z in itertools.product(range(cardA), range(cardM), range(cardY), range(cardZ)):
    model.addConstr( Q_AzAz_MaMa_YmYm[Q_pos_from_P(a,m,y,z)].sum() == P_AMY_giv_Z[(a,m,y,z)], name=f"{P_AMY_giv_Z[(a,m,y,z)]} = P_AMY|Z[{a,m,y}|{z}] = ΣQ{print_pretty(a,m,y,z)}")

In [12]:
model.update()
model.getConstrs()

[<gurobi.Constr 0.336 = P_AMY|Z[(0, 0, 0)|0] = ΣQ[0, 'a', 0, 'm', 0, 'y']>,
 <gurobi.Constr 0.144 = P_AMY|Z[(0, 0, 0)|1] = ΣQ['a', 0, 0, 'm', 0, 'y']>,
 <gurobi.Constr 0.084 = P_AMY|Z[(0, 0, 1)|0] = ΣQ[0, 'a', 0, 'm', 1, 'y']>,
 <gurobi.Constr 0.036 = P_AMY|Z[(0, 0, 1)|1] = ΣQ['a', 0, 0, 'm', 1, 'y']>,
 <gurobi.Constr 0.056 = P_AMY|Z[(0, 1, 0)|0] = ΣQ[0, 'a', 1, 'm', 'y', 0]>,
 <gurobi.Constr 0.024 = P_AMY|Z[(0, 1, 0)|1] = ΣQ['a', 0, 1, 'm', 'y', 0]>,
 <gurobi.Constr 0.224 = P_AMY|Z[(0, 1, 1)|0] = ΣQ[0, 'a', 1, 'm', 'y', 1]>,
 <gurobi.Constr 0.096 = P_AMY|Z[(0, 1, 1)|1] = ΣQ['a', 0, 1, 'm', 'y', 1]>,
 <gurobi.Constr 0.096 = P_AMY|Z[(1, 0, 0)|0] = ΣQ[1, 'a', 'm', 0, 0, 'y']>,
 <gurobi.Constr 0.224 = P_AMY|Z[(1, 0, 0)|1] = ΣQ['a', 1, 'm', 0, 0, 'y']>,
 <gurobi.Constr 0.024 = P_AMY|Z[(1, 0, 1)|0] = ΣQ[1, 'a', 'm', 0, 1, 'y']>,
 <gurobi.Constr 0.056 = P_AMY|Z[(1, 0, 1)|1] = ΣQ['a', 1, 'm', 0, 1, 'y']>,
 <gurobi.Constr 0.036 = P_AMY|Z[(1, 1, 0)|0] = ΣQ[1, 'a', 'm', 1, 'y', 0]>,
 <gurobi.Con

## Do cond

$P_{Y,M|\text{do}(A)}(y,m|\text{do}(a)) = Q(Y_{M=m}=y, M_{A=a}=m)$ from $Q(A_0,A_1,M_0,M_1,Y_0,Y_1)$

done by:
$Q(Y_{M=m}=y, M_{A=a}=m) = \sum_{A_0, A_1, M_{A \not = a}, Y_{M \not = m}} Q(A_0,A_1,M_0,M_1,Y_0,Y_1)$ with given fixed values $(y,m,a)$

In [13]:
# coordinate finder used for P_YM_doA[y,m,a] = Σ_(A0, A1, M_A\not=a, Y_M\not=m)[Q_(a,a',m,m',y,y')]
def do_pos(y,m,a):
    # sliced means iterate through in the sum
    
    # keep M such that M_A=m, slice other
    if a == 0:
        posM0 = m
        posM1 = slice(None)
    elif a == 1:    
        posM0 = slice(None)
        posM1 = m


    if m == 0:
        posY0 = y
        posY1 = slice(None)
    elif m == 1:
        posY0 = slice(None)
        posY1 = y


    # Creating a tuple that properly represents the slicing
    return (slice(None), slice(None), posM0, posM1, posY0, posY1) 

# Confirming:
# str(Q_AzAz_MaMa_YmYm[:, :, :, 1, :, 0]) == str(Q_AzAz_MaMa_YmYm[do_pos(0,1,1)])
# str(Q_AzAz_MaMa_YmYm[:, :, 0, :, 1, :]) == str(Q_AzAz_MaMa_YmYm[do_pos(1,0,0)])


# works by listing symbolic variable name for the variables that are iterated through in the same for all possibilities,
# and an actual value for the fixed variables
def print_pretty2(y,m,a):   
    if a == 0:
        posM = 2
    elif a == 1:
        posM = 3

    if m == 0:
        posY = 4
    elif m == 1:
        posY = 5

    lst = ['a0', 'a1', 'm', 'm', 'y', 'y']
    lst[posM] = m
    lst[posY] = y

    return lst

do conditionals $P_{Y,M|\text{do}(A)}$ and $P_{Y|\text{do}(A)}$ from $Q(A_0,A_1,M_0,M_1,Y_0,Y_1)$:

In [14]:
# do conditional P_YM_doA from Q_AzAz_MaMa_YmYm
def P_YM_doA(y, m, a):
    return Q_AzAz_MaMa_YmYm[do_pos(y,m,a)].sum() # the do_pos slices AzAz out and choses Ma based on a

def P_Y_doA(y, a):
    # P_YM_doA(y, 0, a) + P_YM_doA(y, 1, a)
    return (Q_AzAz_MaMa_YmYm[do_pos(y,0,a)] + Q_AzAz_MaMa_YmYm[do_pos(y,1,a)]).sum().item()

## Seperability

now need $Q(A_0, A_1, M_0, M_1) = Q(A_0, A_1) Q(M_0, M_1)$

$Q(A_0, A_1, M_0, M_1) = \sum_{Y_0, Y_1} Q(A_0,A_1,M_0,M_1,Y_0,Y_1)$  
$Q(A_0, A_1) = \sum_{M_0,M_1,Y_0,Y_1} Q(A_0,A_1,M_0,M_1,Y_0,Y_1)$  
$Q(M_0, M_1) = \sum_{A_0,A_1,Y_0,Y_1} Q(A_0,A_1,M_0,M_1,Y_0,Y_1)$

In [15]:
def Q_A0_A1_M0_M1(A0, A1, M0, M1):
    return Q_AzAz_MaMa_YmYm[A0, A1, M0, M1, :, :].sum().item()

def Q_A0_A1(A0, A1):
    return Q_AzAz_MaMa_YmYm[A0, A1, :, :, :, :].sum().item()

def Q_M0_M1(M0, M1):
    return Q_AzAz_MaMa_YmYm[:, :, M0, M1, :, :].sum().item()

In [16]:
# seperability constraints
for a0, a1, m0, m1 in itertools.product(range(cardA), range(cardA), range(cardM), range(cardM)):
    # MQuadExpr 
    model.addConstr(Q_A0_A1_M0_M1(a0, a1, m0, m1) == Q_A0_A1(a0, a1)*Q_M0_M1(m0, m1), name=f"Q_A0_A1_M0_M1[{a0},{a1},{m0},{m1}] = Q_A0_A1[{a0},{a1}] * Q_M0_M1[{m0},{m1}]")

model.update()

$P_{Y\text{do}A}(a,y) == P_{Y,M_\text{do}A}(y,0,a) + P_{Y,M_\text{do}A}(y,1,a)$ all defined from $Q_{..}$

In [17]:
for a,y in np.ndindex(cardA, cardY):
    model.addConstr(P_Y_doA(a,y) == P_YM_doA(y,0,a) + P_YM_doA(y,1,a), name=f"P_Y_doA[{a},{y}] = P_YM_doA[{y},0,{a}] + P_YM_doA[{y},1,{a}]")

model.update()

## Model Constraints:

Linear:

In [18]:
model.getConstrs()

[<gurobi.Constr 0.336 = P_AMY|Z[(0, 0, 0)|0] = ΣQ[0, 'a', 0, 'm', 0, 'y']>,
 <gurobi.Constr 0.144 = P_AMY|Z[(0, 0, 0)|1] = ΣQ['a', 0, 0, 'm', 0, 'y']>,
 <gurobi.Constr 0.084 = P_AMY|Z[(0, 0, 1)|0] = ΣQ[0, 'a', 0, 'm', 1, 'y']>,
 <gurobi.Constr 0.036 = P_AMY|Z[(0, 0, 1)|1] = ΣQ['a', 0, 0, 'm', 1, 'y']>,
 <gurobi.Constr 0.056 = P_AMY|Z[(0, 1, 0)|0] = ΣQ[0, 'a', 1, 'm', 'y', 0]>,
 <gurobi.Constr 0.024 = P_AMY|Z[(0, 1, 0)|1] = ΣQ['a', 0, 1, 'm', 'y', 0]>,
 <gurobi.Constr 0.224 = P_AMY|Z[(0, 1, 1)|0] = ΣQ[0, 'a', 1, 'm', 'y', 1]>,
 <gurobi.Constr 0.096 = P_AMY|Z[(0, 1, 1)|1] = ΣQ['a', 0, 1, 'm', 'y', 1]>,
 <gurobi.Constr 0.096 = P_AMY|Z[(1, 0, 0)|0] = ΣQ[1, 'a', 'm', 0, 0, 'y']>,
 <gurobi.Constr 0.224 = P_AMY|Z[(1, 0, 0)|1] = ΣQ['a', 1, 'm', 0, 0, 'y']>,
 <gurobi.Constr 0.024 = P_AMY|Z[(1, 0, 1)|0] = ΣQ[1, 'a', 'm', 0, 1, 'y']>,
 <gurobi.Constr 0.056 = P_AMY|Z[(1, 0, 1)|1] = ΣQ['a', 1, 'm', 0, 1, 'y']>,
 <gurobi.Constr 0.036 = P_AMY|Z[(1, 1, 0)|0] = ΣQ[1, 'a', 'm', 1, 'y', 0]>,
 <gurobi.Con

Quadratic:

In [19]:
model.getQConstrs()

[<gurobi.QConstr Q_A0_A1_M0_M1[0,0,0,0] = Q_A0_A1[0,0] * Q_M0_M1[0,0]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[0,0,0,1] = Q_A0_A1[0,0] * Q_M0_M1[0,1]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[0,0,1,0] = Q_A0_A1[0,0] * Q_M0_M1[1,0]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[0,0,1,1] = Q_A0_A1[0,0] * Q_M0_M1[1,1]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[0,1,0,0] = Q_A0_A1[0,1] * Q_M0_M1[0,0]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[0,1,0,1] = Q_A0_A1[0,1] * Q_M0_M1[0,1]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[0,1,1,0] = Q_A0_A1[0,1] * Q_M0_M1[1,0]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[0,1,1,1] = Q_A0_A1[0,1] * Q_M0_M1[1,1]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[1,0,0,0] = Q_A0_A1[1,0] * Q_M0_M1[0,0]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[1,0,0,1] = Q_A0_A1[1,0] * Q_M0_M1[0,1]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[1,0,1,0] = Q_A0_A1[1,0] * Q_M0_M1[1,0]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[1,0,1,1] = Q_A0_A1[1,0] * Q_M0_M1[1,1]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[1,1,0,0] = Q_A0_A1[1,1] * Q_M0_M1[0,0]>,
 <gurobi.QConstr Q_A0_A1_M0_M1[1,1,0,1] = Q_A0_A1[1,1] * Q_M0_M1

## Execution:

In [20]:
(y,b) = (1,1)

In [21]:
%%capture

objective = P_Y_doA(y,b)
model.setObjective(objective, sense=GRB.MINIMIZE)
model.optimize()

minimal = P_Y_doA(y,b).getValue()


objective = P_Y_doA(y,b)
model.setObjective(objective, sense=GRB.MAXIMIZE)
model.optimize()

maximal = P_Y_doA(y,b).getValue()

In [22]:
print("min:", minimal, "\nmax:", maximal, "\nrange:", maximal-minimal)

min: 0.3919999960001179 
max: 0.692 
range: 0.300000003999882


In [23]:
# model.computeIIS()
# model.write("model.ilp")